## 0. Please modify the fig title in the section 4

## 1. Folder init ()

In [ ]:
params = [1.0, 0.1, 0.5]
bv = True
dep = True
knn = False
grouping = False

In [ ]:
import re
import csv
import os
import pandas as pd

nn = 1000 # for synthetic data size
n_zvecs = 100
batch_size = [2, 5, 10, 20, 40]
batch_count = [int(nn/bs) for bs in batch_size]
exa_path = './synthetic_ds'

folder_names = [exa_path]
for bs in batch_size:
    if dep:
        folder_names.append("./batchsize_" + str(bs)+ "_" + str(bs))
    else:
        folder_names.append("./batchsize_" + str(bs)+ "_0")

for folder_name in folder_names:
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    else:
        print(f"Folder '{folder_name}' already exists.")

folder_name_sum = './outputs_sum'
if not os.path.exists(folder_name_sum):
    os.makedirs(folder_name_sum)
    print(f"Folder '{folder_name_sum}' created successfully.")
else:
    print(f"Folder '{folder_name_sum}' already exists.")

In [ ]:
[nn/bs for bs in batch_size]

## 2. log data preprocessing (ExaGeostat)

### 2.2 log file summary

In [ ]:
exa_files = [os.path.join(exa_path, 'log_1000_univariate_matern_stationary_') + str(i) for i in range(1, n_zvecs+1)] # 50 is the replicates
data = []
# Open the input file and create a CSV output file
for i, exa_file in enumerate(exa_files):
    with open(exa_file, 'r') as f:
        lines = f.readlines()
        last_three_lines = lines[-4:]
        for line in last_three_lines:
            if 'Total Number of Iterations=' in line:
                iterations = int(line.split('=')[1])
            elif 'Total Optimization Time= ' in line:
                time = float(line.split('=')[1].strip().split(' ')[0])
            elif 'Found Maximum at (' in line:
                max_values = tuple(map(float, line.split('(')[1].split(',')[:3]))
            elif 'LogLi:' in line:
                llh = float(line.split(':')[1])
    data.append([iterations, time] + list(max_values) + [llh])

# Write the extracted information to a CSV file
with open(os.path.join(folder_name_sum, 'output_full.csv'), 'w', newline='') as f:
    writer = csv.writer(f)
    # Write the header row
    writer.writerow(['Iterations', 'Time', 'variance', 'range', 'smoothness', 'log-likelihood'])
    # Write the data rows
    writer.writerows(data)

# Read data from CSV file
data = pd.read_csv(os.path.join(folder_name_sum, 'output_full.csv'))

# Convert data to pandas dataframe
df = pd.DataFrame(data)

# Save dataframe to original file
df.to_csv(os.path.join(folder_name_sum, 'output_ExaGeoStat.csv'), index=False)

In [ ]:
# data = pd.read_csv('./outputs_sum/output_exact.csv', header=None, index_col=None, sep=' ', \
#     names = ['ine',  'variance', 'range', 'smoothness', 'Time', 'Iterations', 'log-likelihood'])
# data.to_csv('./outputs_sum/output_ExaGeoStat.csv', index=False)

## 3. bash file init for vecchia

### 3.1 bash file created

There are some args to be noticed, such as tolerance, batchCount, etc!

In [ ]:
num_bs = len(batch_size)

with open("run_script.sh", "w") as f:
    for i in range(1, n_zvecs + 1):
        for j in range(num_bs):
            # construct the command line string
            if dep:
                cmd = f"./bin/test_dvecchia_batch -N {batch_size[j]}:1 -s --batchCount {batch_count[j]} --vecchia --maxiter 2000 --kernel 1 --num_loc 1000 --zvecs {i} --tol 9\n"
            else:
                cmd = f"./bin/test_dvecchia_batch -N {batch_size[j]}:1 -s --batchCount {batch_count[j]} --maxiter 2000 --kernel 1 --num_loc 1000 --zvecs {i} --tol 9\n"
            f.write(cmd)

Next, you need to go run `bash ./data/run_script.sh` in the `~/testing`

### 3.2 Summary log summerized

In [ ]:
import pandas as pd
import glob

def read_files_and_concatenate(file_pattern, column_names, output_file):
    """
    Read multiple CSV files with the same format and concatenate them into a single DataFrame.
    
    Parameters:
    file_pattern (str): The file pattern to match, e.g. 'data_*.csv'
    column_names (list of str): The column names for the output DataFrame
    output_file (str): The filename to save the output DataFrame to
    
    Returns:
    None
    """
    # Create an empty DataFrame to hold the data
    df = pd.DataFrame(columns=column_names)

    # Iterate over all matching files
    for filename in glob.glob(file_pattern):
        # Load the current file into a DataFrame
        temp_df = pd.read_csv(filename)
        temp_df.columns = column_names
        # Append the data from the current file to the main DataFrame
        df = pd.concat([df, temp_df])

    # Save the main DataFrame to a new file
    df.to_csv(output_file, index=False)

In [ ]:
# Create an empty DataFrame to hold the data
col_name = ['Iterations', 'Time', 'variance', 'range', 'smoothness', 'log-likelihood']
if bv:
    if dep:
        file_pattern = [f'batchsize_{bs}_{bs}/sum_1000_{bs}_*.csv' for bs in batch_size]
    else:
        file_pattern = [f'batchsize_{bs}_0/sum_1000_{bs}_*.csv' for bs in batch_size]
else:
    file_pattern = [f'batchsize_{bs}_{bs}/sum_1000_{bs}_*.csv' for bs in batch_size]
output_file = [os.path.join(folder_name_sum, f'output_{bs}.csv') for bs in batch_size]
for k in range(len(batch_size)):
    read_files_and_concatenate(file_pattern[k], col_name, output_file[k])
output_file.append('./outputs_sum/output_ExaGeoStat.csv')

## 4. Result visulization

In [ ]:
fig_path = './fig_1K'
if os.path.exists(fig_path):
    print(fig_path + ' exists!')
else:
    os.mkdir(fig_path)
    print(fig_path + ' is created successfully!')

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import csv

# Set the default font size
mpl.rcParams['font.size'] = 14
n_replicates = 100

def plot_time_boxplot(file_names, indic):
    """
    Plot a boxplot of the 'Time' column for each CSV file in the list of file names.
    """
    # Create an empty list to store the time data
    time_data = []

    # Loop through the file names and extract the "Time" column from each CSV file
    for file_name in file_names:
        df = pd.read_csv(file_name)
        if indic != 'combined':
            time_data.append(df[indic])
        else:
            time_data.append(df['variance'].pow(2).div(df['range'].pow(df['smoothness'].mul(2))))
    # Create a boxplot of the time data with x-axis ticks for each file name
    bp = plt.boxplot(time_data, 
                    patch_artist=True,
                    # notch=True, 
                    widths=0.3)
    
    # Set the boxplot colors
    if bv:
        if dep:
            colors = plt.cm.Blues(np.linspace(0.1, 0.9, len(batch_size) + 1))
        else:
            colors = plt.cm.Purples(np.linspace(0.1, 0.9, len(batch_size) + 1))
    else:
        if grouping:
            colors = plt.cm.Reds(np.linspace(0.1, 0.9, len(batch_size) + 1))
        elif knn:
            colors = plt.cm.Oranges(np.linspace(0.1, 0.9, len(batch_size) + 1))
        else:
            colors = plt.cm.Greens(np.linspace(0.1, 0.9, len(batch_size) + 1))
        

    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        
    # Set the line width for the median line
    for median in bp['medians']:
        median.set_linewidth(2)
        
    # Set the line width for the whiskers
    for whisker in bp['whiskers']:
        whisker.set_linewidth(1.5)
    plt.xticks(range(1, len(file_names) + 1), [2, 5, 10, 20, 40, 'Exact'])
    # plt.xticks(range(1, len(file_names) + 1), [20, 40, 50, 100, 200, 250, 500])

    # Add a title and axis labels
    if indic == 'Time':
        # plt.title('Average time ('+ str(n_replicates) + ' replicates)')
        # plt.yscale('log')
        plt.ylabel('Seconds')
    elif indic == 'Iterations':
        # plt.title('Boxplot of total iterations ('+ str(n_replicates) + ' replicates)')
        # plt.yscale('log')
        # plt.ylim(0, 450)
        plt.ylabel('Iterations')
    elif indic == 'variance':
        # plt.title(r'Boxplot of $\hat\sigma$ ('+ str(n_replicates) + ' replicates)')
        # Add a horizontal line at y=0.5 and a text label 'true'
        plt.axhline(y=params[0], color='red', linestyle='--', label=r'$\sigma=$' + str(params[0]))
        # plt.ylim(0.3, 2.5)
        # time_data_mean = np.abs(np.median(time_data, axis=1) - params[0])
        # Add a legend
        # plt.legend(loc='upper left')
    elif indic == 'range':
        # plt.title(r'Boxplot of $\hat\beta$ ('+ str(n_replicates) + ' replicates)')
        # plt.ylim(0.04, 0.20)
        # Add a horizontal line at y=0.5 and a text label 'true'
        plt.axhline(y=params[1], color='red', linestyle='--', label=r'$\beta=$' + str(params[1]))
        # time_data_mean = np.abs(np.median(time_data, axis=1) - params[1])
        # Add a legend
        # plt.legend(loc='upper left')
    elif indic == 'smoothness':
        # plt.title(r'Boxplot of $\hat\nu$ ('+ str(n_replicates) + ' replicates)')
        # plt.ylim(0.20, 1.00)
        # Add a horizontal line at y=0.5 and a text label 'true'
        plt.axhline(y=params[2], color='red', linestyle='--', label=r'$\nu=$' + str(params[2]))
        # time_data_mean = np.abs(np.median(time_data, axis=1) - params[2])
        # Add a legend
        # plt.legend(loc='upper left')
    # elif indic == 'log-likelihood':
    #     plt.title(r'Boxplot of log-likelihood ('+ str(n_replicates) + ' replicates)')
    #     # plt.ylim(-900, 2800)
    
    elif indic == 'combined':
        plt.title(r'Boxplot of $\frac{\hat\sigma^2}{\hat\beta^{2\hat\nu}}$ ('+ str(n_replicates) + ' replicates)')
        # plt.ylim(0, 30)
        # Add a horizontal line at y=0.5 and a text label 'true'
        cons_esti = params[0]**2 / (params[1]**(2*params[2]))
        plt.axhline(y=cons_esti, color='red', linestyle='--', label=r'$\frac{\sigma^2}{\beta^{2\nu}} = $' + str(round(cons_esti, 2)))
        # time_data_mean = np.abs(np.median(time_data, axis=1) - cons_esti)
        # Add a legend
        # plt.legend(loc='upper left')

    # Show the plot
    plt.legend(loc='upper left')

    # title
    if not bv:
        if grouping:
            plt.title('GpGp (KNN and Grouping)')
        elif knn:
            plt.title('GpGp (KNN)')
        else:
            plt.title('GpGp (base)')
        ## Save the difference of the data
        ## Used for quantitve description 
        # with open('./1k_' + str(params[1]) + '_' + str(params[2]) + str('_gpgp.csv')
        #         , mode='a', newline='') as file:
        #     writer = csv.writer(file)
        #     writer.writerow(time_data_mean)
    else:
        if dep:
            plt.title('BV (dependent)')
        else:
            plt.title('BV (independent)')
        ## Save the difference of the data
        ## Used for quantitve description 
        # with open('./1k_' + str(params[1]) + '_' + str(params[2]) + str('_bv.csv')
        #         , mode='a', newline='') as file:
        #     writer = csv.writer(file)
        #     writer.writerow(time_data_mean)
    plt.xlabel('The number of neighbors')
    plt.savefig(os.path.join(fig_path, indic) + '_1k_' + str(params[1]) + '_' + str(params[2]) + str('.pdf'), 
                bbox_inches='tight', format = 'pdf')
    plt.show()


In [ ]:
# output_file = [os.path.join(folder_name_sum, f'output_{bs}.csv') for bs in batch_size_all]
# plot_time_boxplot(output_file, indic='Time')

In [ ]:
plot_time_boxplot(output_file, indic='Iterations')

In [ ]:
# plot_time_boxplot(output_file, indic='log-likelihood')

In [ ]:
plot_time_boxplot(output_file, indic='combined')

In [ ]:
plot_time_boxplot(output_file, indic='variance')

In [ ]:
plot_time_boxplot(output_file, indic='range')

In [ ]:
plot_time_boxplot(output_file, indic='smoothness')